### Adjust Word Group Repeat With Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [9]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

In [10]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [11]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/3-Adjust Word Group Repeat With Youtube Link"

#Path(path).mkdir(parents=True, exist_ok=True)

In [12]:
path_folder_file = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish/Twogram_Threegram_Selected.xlsx"
sheet = "twogram_threegram_selected"  # Sheet1

In [13]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")  
df_word_group_time_loc

,twogram_threegram_selected
0,benimle geliyor musun
1,bana yardım et
2,sadece birkaç dakika
3,birkaç gün içinde
4,ben teşekkür ederim
...,...
95,onun gibi
96,para için
97,senin için
98,sorun yok


In [17]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

100

In [151]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [ ]:
# All word group result convert to join result in one row
df_result_repeat = pd.DataFrame()
for word_group in search_list:
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)

df_result_repeat.reset_index(drop=True, inplace=True)
df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])

df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time.xlsx", index=False)
df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join.xlsx", index=False)

In [ ]:
df_result_repeat

In [ ]:
(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [155]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=1)

df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join.xlsx", index=False)


In [156]:
df_all_join_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,is dat zo,603,605,en op moment dat ik het een keer verkeerd loop...,VjYg2X32ryM,https://www.youtube.com/watch?v=VjYg2X32ryM&t=...
1,is dat zo,95,97,wat is er aan de hand in iran en waarom is dat...,ypikak3H9MI,https://www.youtube.com/watch?v=ypikak3H9MI&t=95s
2,is dat zo,623,624,is dat zo scherp bij transport van mest,USIYNJg2qSw,https://www.youtube.com/watch?v=USIYNJg2qSw&t=...
3,is dat zo,17,19,is dat zo ja ik werd vanochtend wakker om zes ...,NMUAd5R67K0,https://www.youtube.com/watch?v=NMUAd5R67K0&t=17s
4,is dat zo,73,74,is dat zo zijn jullie altijd zo rustig,cUB3dxLXvSc,https://www.youtube.com/watch?v=cUB3dxLXvSc&t=73s
...,...,...,...,...,...,...
112,repeat,5,8,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s
113,weet ik niet,279,281,ja weet ik niet maar ik denk dus dat ik onderd...,GE9qSjs18og,https://www.youtube.com/watch?v=GE9qSjs18og&t=...
114,repeat,5,8,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s
115,weet ik niet,279,281,ja weet ik niet maar ik denk dus dat ik onderd...,GE9qSjs18og,https://www.youtube.com/watch?v=GE9qSjs18og&t=...


#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("Twogram Threegram Youtube Link Selected.xlsx")
df

,search_string,start_time,end_time,video_url,id
0,benimle geliyor musun,6543,6545,https://www.youtube.com/watch?v=liz1ODAVzyk&t=...,liz1ODAVzyk
1,bana yardım et,254,257,https://www.youtube.com/watch?v=Iw1u6ZYA4t8&t=...,Iw1u6ZYA4t8
2,bana yardım et,3392,3395,https://www.youtube.com/watch?v=OTDkvCGQyVQ&t=...,OTDkvCGQyVQ
3,bana yardım et,4017,4020,https://www.youtube.com/watch?v=y3rhd_QEgck&t=...,y3rhd_QEgck
4,sadece birkaç dakika,1643,1645,https://www.youtube.com/watch?v=6Pr093ZQF5A&t=...,6Pr093ZQF5A
...,...,...,...,...,...
447,tamam mı,5776,5778,https://www.youtube.com/watch?v=oCsHNgql560&t=...,oCsHNgql560
448,tamam mı,2315,2317,https://www.youtube.com/watch?v=oCsHNgql560&t=...,oCsHNgql560
449,tamam mı,209,212,https://www.youtube.com/watch?v=xHyS_6Hq5GA&t=...,xHyS_6Hq5GA
450,tamam mı,558,559,https://www.youtube.com/watch?v=W_ofzCKTKvY&t=...,W_ofzCKTKvY


In [3]:
df_count = df.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,asla olmaz,7
1,birkaç gün,7
2,neden böyle oldu,7
3,aynı şekilde,7
4,hadi bakalım,7
...,...,...
91,benimle geliyor musun,1
92,hayır sorun yok,1
93,lütfen izin ver,1
94,orada mısın sen,1


In [4]:
df_count_merge = pd.merge(df, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,video_url,id,count
0,önemli olan,497,499,https://www.youtube.com/watch?v=NTG7NZgQals&t=...,NTG7NZgQals,7
1,önemli olan,1726,1728,https://www.youtube.com/watch?v=vsDoBhQJ1hY&t=...,vsDoBhQJ1hY,7
2,önemli olan,6246,6248,https://www.youtube.com/watch?v=m3Clq7uulPg&t=...,m3Clq7uulPg,7
3,önemli olan,406,408,https://www.youtube.com/watch?v=kzEvSIem_n4&t=...,kzEvSIem_n4,7
4,önemli olan,6184,6186,https://www.youtube.com/watch?v=JV6ZUdw_lXQ&t=...,JV6ZUdw_lXQ,7
...,...,...,...,...,...,...
447,orada mısın sen,6253,6255,https://www.youtube.com/watch?v=IDBLfpy15Jk&t=...,IDBLfpy15Jk,1
448,lütfen izin ver,511,513,https://www.youtube.com/watch?v=nTYDDNdrdZM&t=...,nTYDDNdrdZM,1
449,hayır sorun yok,781,783,https://www.youtube.com/watch?v=iSIdXpAd76I&t=...,iSIdXpAd76I,1
450,biliyorsun değil mi,5759,5761,https://www.youtube.com/watch?v=O5SvwYGbXCE&t=...,O5SvwYGbXCE,1


In [13]:
df_count_merge["search_string"].nunique()

96

In [12]:
df_count_merge[df_count_merge["count"] == 1]

,search_string,start_time,end_time,video_url,id,count
446,yardım ister misin,1947,1950,https://www.youtube.com/watch?v=SUfgeDos2Kw&t=...,SUfgeDos2Kw,1
447,orada mısın sen,6253,6255,https://www.youtube.com/watch?v=IDBLfpy15Jk&t=...,IDBLfpy15Jk,1
448,lütfen izin ver,511,513,https://www.youtube.com/watch?v=nTYDDNdrdZM&t=...,nTYDDNdrdZM,1
449,hayır sorun yok,781,783,https://www.youtube.com/watch?v=iSIdXpAd76I&t=...,iSIdXpAd76I,1
450,biliyorsun değil mi,5759,5761,https://www.youtube.com/watch?v=O5SvwYGbXCE&t=...,O5SvwYGbXCE,1
451,benimle geliyor musun,6543,6545,https://www.youtube.com/watch?v=liz1ODAVzyk&t=...,liz1ODAVzyk,1


In [5]:
df_count_merge.to_excel("Twogram Threegram Youtube Link Selected_Count.xlsx")